In [31]:
from requests import get
from json import loads
from services.db_service import DbService

In [32]:
db_service = DbService()

In [33]:


def get_card_scryfall_name_and_color(name):
    card = loads(
        get(f"https://api.scryfall.com/cards/named?exact={name}").text)

    type = card["type_line"]
    name = card["name"]
    legality = card["legalities"]["pauper"]

    if legality == "not_legal" or type == "Stickers":
        return name, "I"

    if "Land" in type:
        return name, "L"

    if "colors" in card:
        colors = card["colors"]
    else:
        colors = card["card_faces"][0]["colors"]

    if len(colors) == 0:
        return name, "C"
    elif len(colors) > 1:
        return name, "M"
    else:
        return name, colors[0]

In [34]:
pool_collection = db_service.pool_collection()


def add_card_or_update(card):
    name = card["name"]
    scryfall_name, color = get_card_scryfall_name_and_color(name)

    if color == "I":
        return

    existing_card = pool_collection.find_one({"name": name})
    if existing_card and existing_card["count"] == card["count"]:
        return

    if existing_card and existing_card["count"] != card["count"]:
        pool_collection.update_one({"_id": existing_card["_id"]}, {
                                   "$set": {"count": card["count"]}})
        return

    pool_collection.insert_one(
        {"name": name, "count": card["count"], "color": color, "scryfall_name": scryfall_name})

In [35]:
aggregate_query = [
    {"$match": {"restricted": {"$ne": True}}},
    {"$project": {"allCards": {"$concatArrays": ["$maindeck", "$sideboard"]}}},
    {"$unwind": "$allCards"},
    {"$group": {"_id": {"deckId": "$_id", "name": "$allCards"}, "count": {"$sum": 1}}},
    {"$group": {"_id": "$_id.name", "count": {"$max": "$count"}}},
    {"$sort": {"_id": 1}},
    {"$project": {"name": "$_id", "count": "$count", "_id": 0}}
]

In [36]:
deck_collection = db_service.deck_collection()

cards = list(deck_collection.aggregate(aggregate_query))
for card in cards:
    if card["name"] == "Unknown Card":
        continue

    add_card_or_update(card)